######  Apache Spark™ is a unified analytics engine for large-scale data processing. Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance. On Single Machine , Spark can have lower memory consumption and can process more data than laptop ’s memory size, as it does not require loading the entire data set into memory before processing

#### Intialize context to interact with Spark Cluster

In [10]:
from pyspark.sql import SparkSession


In [11]:
spark = SparkSession.builder \
...     .master("local") \
...     .appName("Word Count") \
...     .getOrCreate()

In [12]:
dir(spark)

['Builder',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_convert_from_pandas',
 '_createFromLocal',
 '_createFromRDD',
 '_create_from_pandas_with_arrow',
 '_create_shell_session',
 '_get_numpy_record_dtype',
 '_inferSchema',
 '_inferSchemaFromList',
 '_instantiatedSession',
 '_jsc',
 '_jsparkSession',
 '_jvm',
 '_jwrapped',
 '_repr_html_',
 '_sc',
 '_wrapped',
 'builder',
 'catalog',
 'conf',
 'createDataFrame',
 'newSession',
 'range',
 'read',
 'readStream',
 'sparkContext',
 'sql',
 'stop',
 'streams',
 'table',
 'udf',
 'version']

In [15]:
spark.version

'2.4.3'

### Read Log file and Perform Preprocessing

In [16]:
log_file_path = r'D:\jupyter bootcamp\clonedrepo\notebooks\testdata\apache_logs\apache.access.log.PROJECT_2.txt'

In [17]:
base_df = spark.read.text(log_file_path)

In [18]:
base_df.printSchema()

root
 |-- value: string (nullable = true)



In [19]:
base_df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------+
|reports.arc.nasa.gov - - [09/Aug/1995:13:23:23 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713                              |
|atl57298.bwi.wec.com - - [09/Aug/1995:13:23:24 -0400] "GET /shuttle/missions/sts-71/movies/sts-71-launch-3.mpg HTTP/1.0" 200 130724|
|cat.dbs.ucdavis.edu - - [09/Aug/1995:13:23:24 -0400] "GET /shuttle/missions/sts-70/mission-sts-70.html HTTP/1.0" 200 20113         |
|gorgax06.pixi.com - - [09/Aug/1995:13:23:24 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 200 234                                |
|russell.dp.ox.ac.uk - - [09/Aug/1995:13:23:24 -0400] "GET /hi

In [22]:
base_df.describe()

DataFrame[summary: string, value: string]

In [23]:
base_df.count()

350296

In [24]:
from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.show(truncate=False)

+------------------------+--------------------------+---------------------------------------------------+------+------------+
|host                    |timestamp                 |path                                               |status|content_size|
+------------------------+--------------------------+---------------------------------------------------+------+------------+
|reports.arc.nasa.gov    |09/Aug/1995:13:23:23 -0400|/images/launch-logo.gif                            |200   |1713        |
|atl57298.bwi.wec.com    |09/Aug/1995:13:23:24 -0400|/shuttle/missions/sts-71/movies/sts-71-launch-3.mpg|200   |130724      |
|cat.dbs.ucdavis.edu     |09/Aug/1995:13:23:24 -0400|/shuttle/missions/sts-70/mission-sts-70.html       |200   |20113       |
|gorgax06.pixi.com       |09/Aug/1995:13:23:24 -0400|/images/USA-logosmall.gif                          |200   |234         |
|russell.dp.ox.ac.uk     |09/Aug/1995:13:23:24 -0400|/history/apollo/apollo-13/apollo-13.html           |200   |18556 

In [27]:
#check  if new df has any null values
from pyspark.sql.functions import col, sum

def count_null(col_name):
  return sum(col(col_name).isNull().cast('integer')).alias(col_name)

# Build up a list of column expressions, one per column.
#
# This could be done in one line with a Python list comprehension, but we're keeping
# it simple for those who don't know Python very well.
exprs = []
for col_name in split_df.columns:
  exprs.append(count_null(col_name))

# Run the aggregation. The *exprs converts the list of expressions into
# variable function arguments.
split_df.agg(*exprs).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     0|        2677|
+----+---------+----+------+------------+



In [28]:
## fill all null values with 0
cleaned_df = split_df.na.fill({'content_size': 0})

In [29]:
# Ensure that there are no nulls left.
exprs = []
for col_name in cleaned_df.columns:
      exprs.append(count_null(col_name))

cleaned_df.agg(*exprs).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     0|           0|
+----+---------+----+------+------------+



In [43]:
## Handle Timestamp in dataframe
from datetime import datetime as dt
converted= dt.strptime("09/Aug/1995:13:23:23 -0400", "%d/%b/%Y:%H:%M:%S %z")


In [44]:
converted

datetime.datetime(1995, 8, 9, 13, 23, 23, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))

from pyspark.sql.functions import udf
from datetime import datetime as dt

def parse_clf_time(s):
    return dt.strptime(s, "%d/%b/%Y:%H:%M:%S %z")

u_parse_time = udf(parse_clf_time)

logs_df = cleaned_df.select('*', u_parse_time(split_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = logs_df.count()

In [48]:
from pyspark.sql.functions import udf
month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(s):
    """ Convert Common Log time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format [dd/mmm/yyyy:hh:mm:ss (+/-)zzzz]
    Returns:
        a string suitable for passing to CAST('timestamp')
    """
    # NOTE: We're ignoring time zone here. In a production application, you'd want to handle that.
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(s[7:11]),
      month_map[s[3:6]],
      int(s[0:2]),
      int(s[12:14]),
      int(s[15:17]),
      int(s[18:20])
    )

u_parse_time = udf(parse_clf_time)

logs_df = cleaned_df.select('*', u_parse_time(split_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = logs_df.count()

In [49]:
logs_df.printSchema()

root
 |-- host: string (nullable = true)
 |-- path: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- content_size: integer (nullable = false)
 |-- time: timestamp (nullable = true)



In [50]:
total_log_entries

350296

In [51]:
### keep the dataframe in cache for increasing speed
logs_df.cache()

DataFrame[host: string, path: string, status: int, content_size: int, time: timestamp]

In [52]:
# Calculate statistics based on the content size.
content_size_summary_df = logs_df.describe(['content_size'])
content_size_summary_df.show()

+-------+-----------------+
|summary|     content_size|
+-------+-----------------+
|  count|           350296|
|   mean|17778.18860620732|
| stddev| 70095.7214882202|
|    min|                0|
|    max|          3155499|
+-------+-----------------+



In [54]:
status_to_count_df =(logs_df
                     .groupBy('status')
                     .count()
                     .sort('status')
                     .cache())

In [55]:
status_to_count_length = status_to_count_df.count()
print('Found %d response codes' % status_to_count_length)
status_to_count_df.show()

Found 6 response codes
+------+------+
|status| count|
+------+------+
|   200|315078|
|   302|  5759|
|   304| 27650|
|   403|    23|
|   404|  1781|
|   501|     5|
+------+------+



In [56]:
display(status_to_count_df)


DataFrame[status: int, count: bigint]